In [142]:
from langgraph.graph import StateGraph, START , END
from dotenv import load_dotenv
load_dotenv()
from typing import TypedDict,Optional, Annotated,List
from  langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint
from  langchain_core.prompts import PromptTemplate
from  langchain_core.messages import HumanMessage, SystemMessage
from pydantic import BaseModel


In [143]:
llm = HuggingFaceEndpoint(model="meta-llama/Meta-Llama-3-8B-Instruct",task="text-generation")
model = ChatHuggingFace(llm=llm)

In [144]:
from typing import Literal


class TweetState(TypedDict):
    topic: str
    tweet:str
    old_tweet: List[str]
    iteration : int
    max_iteration : int
    feedback : Literal['approved', 'not approved'] | None



In [145]:
def generate_tweet(state: TweetState) ->TweetState:
    topic = state['topic'] 
    
    messages = [
        SystemMessage(content='Make the tweet sharp and short'),
        HumanMessage(content=f'Make a tweet about this topic: {topic}'),
    ]
    
    response = model.invoke(messages)
    
    state['iteration'] += 1 
    if type(response.content)==str:
        state['tweet']=response.content
    return state


In [146]:
class TweetFeedback(BaseModel):
    clarity: int
    brevity : int
    engagement : int
    relevance : int
    overall_score : float
    suggestion : str


In [147]:
def get_feedback(response):
    if response['overall_score'] > 9:
        return 'approved'
    else:
        return 'not approved'

In [148]:
class TweetFeedback(BaseModel):
    clarity: int
    brevity : int
    engagement : int
    relevance : int
    overall_score : float
    suggestion : str

def feedback_tweet(state: TweetState) -> TweetState:
    tweet = state['tweet']

    messages = [
        SystemMessage(content="Rate the tweet from 1–10 on clarity, brevity, engagement, relevance, originality and then overall_score. Return JSON dict only with these keys and there value clarity, brevity, engagement, relevance, originality and then overall_score."),
        HumanMessage(content=f"{tweet}"),
    ]

    response = model.invoke(messages)  

    import json
    feedback_data = json.loads(response.content)
    print(response.content)
    
    state['feedback'] = 'approved' if feedback_data['overall_score'] > 9 else 'not approved'
    return state



In [149]:
def enchanceTweet(state:TweetState)->TweetState:
    tweet = state['tweet'] 
    state['old_tweet'].append(state['tweet'])

    messages = [
        SystemMessage("You are a expert tweeter what you need to do is make the tweet good by these terms clarity, brevity, engagement, relevance, and originality"),
        HumanMessage(content=f'{tweet}'),
    ]

    state['iteration'] = state['iteration'] + 1
    response = model.invoke(messages)
    if type(response.content) == str:
        state['tweet']=response.content

    
    return state

In [150]:
graph = StateGraph(TweetState)


In [151]:
def return_literal(state:TweetState):
    if state['feedback']=='approved' or state['iteration'] > state['max_iteration']:
        return 'approved'
    else:
        return 'not approved'

In [152]:
graph.add_node('generate_tweet',generate_tweet)
graph.add_node('feedback_tweet',feedback_tweet)
graph.add_node('enchanceTweet',enchanceTweet)




graph.add_edge(START,'generate_tweet')
graph.add_edge('generate_tweet','feedback_tweet')
graph.add_conditional_edges('feedback_tweet',return_literal,{
    'approved':END,
    'not approved':"enchanceTweet"
})
graph.add_edge('enchanceTweet','feedback_tweet')


In [153]:
workflow = graph.compile()

In [154]:
# workflow

In [155]:
state: TweetState = {
    'topic': "BJP",
    'max_iteration': 5,
    'iteration': 0,
    'tweet':'',
    'old_tweet':[''],
    'feedback':None
}

result = workflow.invoke(state)


HfHubHTTPError: 402 Client Error: Payment Required for url: https://router.huggingface.co/novita/v3/openai/chat/completions (Request ID: Root=1-697326ba-3a1639cf1e72cef6202dca68;59dae382-4e29-4943-a6df-c423a4c7ba13)

You have reached the free monthly usage limit for novita. Subscribe to PRO to get 20x more included usage, or add pre-paid credits to your account.